# Análisis de *datasets* de la FIFA para que Chile llegue lejos


## Resumen

Chile necesita llegar de buena manera al mundial y en base a un análisis de datasets de la FIFA se podrá ver cuales son las características del juego que más ayudan a las selecciones a ganar. La metodología ocupada fue extraer los datos desde *kagle*, y transformarlo para cada visualización ocupando la herramienta de plotly.
Como conclusiones, se obtuvo que la selección debe tener un juego con menos pose de balón y más táctico, la cantidad de faltas no es determinante, que se debe ser efectivo en el ataque y no compremeter la defensa. Son propuestas que hacen menos vistoso el juego pero han dado resultado y se observa una tendencia de los últimos campeones del mundo en disminuir la cantidad de tiros al arco y aumentar la cantida de tarjetas amarillas.

## Definición del problema
Luego del fracaso de la selección chilena de fútbol por no clasificar al mundial de Rusia, el nuevo técnico de la selección Reinaldo Rueda pretende imponer un nuevo estilo de juego para que la selección tenga mejores resultados, pueda clasificar y con el objetivo final de ganar la copa del mundo en Qatar 2022. Para atacar este problema, se dispone de un *dataset* con las estadísticas de cada partido del pasado mundial, lo que permitirá dar una visión del fútbol actual, además, este dataset se complementará con los partidos de los campeones mundiales del año 2010 y 2014. Se pretende como objetivo, rescatar tácticas o distintas variantes en base al dataset del pasado mundial y ratificarlas con los campeones pasados para entrenar la selección para fortalecer estas variantes.

## Metodología
#### Extracción de datos
Los datos fueron extraídos de [*kaggle*](https://www.kaggle.com/mathan/fifa-2018-match-statistics) del caso : 'Predict FIFA 2018 Man of the Match' el cual pretendía mediante técnicas de *ML*, predecir a que equipo pertenecía el *Man of the Match* (jugador del partido), también de la página oficial de la [FIFA](https://www.fifa.com/fifa-tournaments/statistics-and-records/worldcup/index.html) para los datos de los campeones pasados. Se dejó toda la información en un solo .csv , luego mediante filtros se obtendrán los datos necesarios para cada visualización.
#### Herramienta para el análisis
La librería utilizada para las visualizaciones es plotly.
#### Dificultades encontradas
Se podría considerar una dificultad, la extracción de datos para los campeones pasados, no se tenía la información en un único archivo para facilitar el tratamiento, sino cada partido tenía 10 pdfs con cada aspecto del juego ( Distribución de pases, estadística de los jugadores de ambos equipos, formación, mapa de calor de jugadores en el campo, etc), lo que hizo el proceso engorroso y lento.
Plotly es una buena herramienta para visualizar e interactuar pero restringe la cantidad de gráficos que se pueden hacer, así que hay que crearse una nueva, y realizar el proceso de configurar las credenciales tantas veces se necesite.

## Resultados y Discusión
#### Manejo de datos

In [17]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Lectura
df = pd.read_csv('fifa_statistics.csv',';')
df.info()
print("\nColumnas con espacios vacios")
print(df.columns[df.isna().any()].tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 27 columns):
Date                      156 non-null object
Team                      156 non-null object
Opponent                  156 non-null object
Goal Scored               156 non-null int64
Ball Possession %         156 non-null int64
Attempts                  156 non-null int64
On-Target                 156 non-null int64
Off-Target                156 non-null int64
Blocked                   156 non-null int64
Corners                   156 non-null int64
Offsides                  156 non-null int64
Free Kicks                156 non-null int64
Saves                     156 non-null int64
Pass Accuracy %           156 non-null int64
Passes                    156 non-null int64
Distance Covered (Kms)    128 non-null float64
Fouls Committed           156 non-null int64
Yellow Card               156 non-null int64
Yellow & Red              156 non-null int64
Red                       156 non-n

Respecto a las columnas con espacios vacíos, no se pudo obtener la distancia recorrida de los campeones 2010 y 2014, los otros campos dependen netamente porque no hay goles en el partido, como tampoco hay goles en contra.

In [18]:
# Muestra de datos
df.head(6)

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,...,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time
0,14-06-18,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,0,0,0,Yes,12.0,Group Stage,No,0,NaN,NaN
1,14-06-18,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,0,0,0,No,NaN,Group Stage,No,0,NaN,NaN
2,15-06-18,Egypt,Uruguay,0,43,8,3,3,2,0,...,2,0,0,No,NaN,Group Stage,No,0,NaN,NaN
3,15-06-18,Uruguay,Egypt,1,57,14,4,6,4,5,...,0,0,0,Yes,89.0,Group Stage,No,0,NaN,NaN
4,15-06-18,Morocco,Iran,0,64,13,3,6,4,5,...,1,0,0,No,NaN,Group Stage,No,0,1.0,90.0
5,15-06-18,Iran,Morocco,1,36,8,2,5,1,2,...,3,0,0,Yes,90.0,Group Stage,No,0,NaN,NaN


In [19]:
# Por comodidad, renombrar columnas
df_col = df.columns
df_col_renombradas = ['fecha','equipo','oponente','goles','posesion','intentos','arco','fuera','bloqueado','corners','offsides',
                      'tiro_libre','salvadas','precision_pases','pases','distancia','faltas','amarillas',
                      'amarilla_y_roja','rojas','MoM','primer_gol','ronda','penales','goles_penales',
                      'autogol','autogol_tiempo']
df.rename(columns = dict(zip(df_col,df_col_renombradas)),inplace=True)
pd.set_option('max_colwidth', 1000)
df.head()

,fecha,equipo,oponente,goles,posesion,intentos,arco,fuera,bloqueado,corners,...,amarillas,amarilla_y_roja,rojas,MoM,primer_gol,ronda,penales,goles_penales,autogol,autogol_tiempo
0,14-06-18,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,0,0,0,Yes,12.0,Group Stage,No,0,NaN,NaN
1,14-06-18,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,0,0,0,No,NaN,Group Stage,No,0,NaN,NaN
2,15-06-18,Egypt,Uruguay,0,43,8,3,3,2,0,...,2,0,0,No,NaN,Group Stage,No,0,NaN,NaN
3,15-06-18,Uruguay,Egypt,1,57,14,4,6,4,5,...,0,0,0,Yes,89.0,Group Stage,No,0,NaN,NaN
4,15-06-18,Morocco,Iran,0,64,13,3,6,4,5,...,1,0,0,No,NaN,Group Stage,No,0,1.0,90.0


In [20]:
def dejar_anno(x):
    x = '20'+ x[-2:]
    return x

df.fecha = df.fecha.apply(dejar_anno)

### Posesión del balon promedio por equipo.

In [21]:
# Manejo de datos.
df_2018 = df[df.fecha == '2018']
posesiones = pd.DataFrame()
equipos_2018 = df_2018.groupby('equipo')
equipos = equipos_2018.groups.keys()
for j in equipos:
    partidas = equipos_2018.get_group(j).reset_index()
    partidas = partidas[['equipo','posesion']]
    cantidad_de_partidos = partidas.shape[0]
    posesion = round(partidas.posesion.sum()/cantidad_de_partidos,1)
    if(cantidad_de_partidos==3):
    	situacion = 'Eliminado en grupos'
    	pos = 32
    	color = 'red'
    elif(cantidad_de_partidos==4):
    	situacion = 'Eliminado en 8º'
    	pos = 16
    	color = 'orange'
    elif(cantidad_de_partidos==5):
    	situacion = 'Eliminado en 4º'
    	pos = 8
    	color = 'yellow'
    elif(cantidad_de_partidos==6):
    	situacion = 'Semifinalista'
    	pos = 4
    	color = 'green'
    elif(cantidad_de_partidos==7):
    	if(j=='France'):
    		situacion = 'Campeón'
    		pos = 1
    	elif(j=='Croatia'):
    		situacion = 'Vicecampeón'
    		pos = 2
    	elif(j=='Belgium'):
    		situacion = '3º lugar'
    		pos = 3
    	elif(j=='England'):
    		situacion = '4º lugar'
    		pos = 4
    	color = 'green'
    partidas['promedio_posesion'] = posesion
    partidas['situacion'] = situacion
    partidas['posicion'] = pos
    partidas['color'] = color
    partidas = partidas[['equipo','promedio_posesion','situacion','posicion','color']]
    fila = partidas.loc[0,:]
    posesiones = posesiones.append(fila,ignore_index=True)
posesiones.loc[posesiones.posicion == 8,'posicion'] = range(5,9)
posesiones = posesiones.sort_values("posicion",ascending=True).reset_index(drop=True)

In [22]:
# Visualización - Barras

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

colors = posesiones.color.values

trace = go.Bar(
    x= posesiones.equipo.values,
    y= posesiones.promedio_posesion.values,
    text = posesiones.situacion.values,
    marker=dict(
        color= colors,
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.8
)

data = [trace]

layout = go.Layout(
    title='Promedio de posesión de balón por selección - Mundial 2018',
    xaxis=dict(
        title='Selecciones',
        tickangle=45,
        titlefont=dict(
            size=15,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=12,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Porcentaje promedio de posesión',
        titlefont=dict(
            size=15,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=12,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='text-hover-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~F4BI4N2/0 or inside your plot.ly account where it is named 'text-hover-bar'


##### Comentarios
El orden en el eje x corresponde al puesto que tuvo la selección en el mundial. Se puede observar que el campeón Francia obtuvo el menor promedio de posesión entre los 4 finalistas, que corresponde a un 47,9 %, o sea en promedio, en cada partido jugado, su contricante tuvo más posesión del balón. Evidentemente, la posesión no fue un factor relevante en el ganador final, basta ver las selecciones de Alemania, Argentina y España que tienen altos promedios pero ninguna de estas selecciones pudo pasar a los cuartos de final.

### Agresividad en el juego

In [23]:
# Manejo de datos.
faltas = pd.DataFrame()
for j in equipos:
    partidas = equipos_2018.get_group(j).reset_index()
    partidas = partidas[['equipo','faltas','amarillas','amarilla_y_roja','rojas']]
    cantidad_de_partidos = partidas.shape[0]
    total_faltas = partidas.faltas.sum() 
    total_amarillas = partidas.amarillas.sum()
    total_a_y_r = partidas.amarilla_y_roja.sum()
    total_rojas = partidas.rojas.sum()
    if(cantidad_de_partidos==3):
    	pos = 32
    elif(cantidad_de_partidos==4):
    	pos = 16
    elif(cantidad_de_partidos==5):
    	pos = 8
    elif(cantidad_de_partidos==6):
    	pos = 4
    elif(cantidad_de_partidos==7):
    	if(j=='France'):
    		pos = 1
    	elif(j=='Croatia'):
    		pos = 2
    	elif(j=='Belgium'):
    		pos = 3
    	elif(j=='England'):
    		pos = 4
    partidas = partidas[['equipo']]
    fila = partidas.loc[0,:]
    fila['total_faltas'] = total_faltas
    fila['total_amarillas'] = total_amarillas
    fila['total_a_y_r'] = total_a_y_r
    fila['total_rojas'] = total_rojas
    fila['posicion'] = pos
    faltas = faltas.append(fila,ignore_index=True)
faltas.loc[faltas.posicion == 8,'posicion'] = range(5,9)
faltas = faltas.sort_values("posicion",ascending=True).reset_index(drop=True)

In [37]:
# Visualización - Scatter plot o podria haber sido Stacked bars
import numpy as np
faltas_g = faltas.total_faltas.values
amarillas = faltas.total_amarillas.values.tolist()
rojas = faltas.total_rojas.values.tolist()
rojas = [x if (x != 0) else '' for x in rojas]
doble_amarilla = faltas.total_a_y_r.values.tolist()
doble_amarilla = [x if (x != 0) else '' for x in doble_amarilla]
equipos = faltas.equipo.values

# Create traces
trace0 = go.Scatter(
    x = equipos,
    y = faltas_g,
    mode = 'lines+markers',
    name = 'Total de faltas', 
    marker = dict(
        line = dict(
            width = 5,
            color = 'blue'
        )
    )
)
trace1 = go.Scatter(
    x = equipos,
    y = amarillas,
    mode = 'markers',
    name = 'Amarillas',
    marker = dict(
        size = 10,
        color = 'yellow',
        line = dict(
            width = 1,
            color = 'rgb(0, 0, 0)'
        )
    )

)
trace2 = go.Scatter(
    x = equipos,
    y = rojas,
    mode = 'markers',
    name = 'Rojas',
    marker = dict(
        size = 10,
        color = 'red',
        line = dict(
            width = 1,
            color = 'rgb(0, 0, 0)'
        )
    )
)
trace3 = go.Scatter(
    x = equipos,
    y = doble_amarilla,
    mode = 'markers',
    name = 'Dobles amarillas',
    marker = dict(
        size = 10,
        color = 'orange',
        line = dict(
            width = 1,
            color = 'rgb(0,0,0)'
        )
    )
)

layout = go.Layout(
    title='Cantidad de faltas por equipo - Mundial 2018',
    xaxis=dict(
        title='Selecciones',
        tickangle=55,
        titlefont=dict(
            size=15,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=12,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Cantidad de infracciones',
        titlefont=dict(
            size=15,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=12,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        x=1,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    bargap=0.15,
    bargroupgap=0.1
)

data = [trace0,trace1,trace2,trace3]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='scatter-mode')

##### Comentarios
Los 4 primeros puestos son los que tienen más faltas ya que jugaron más partidos que las otras selecciones, de aquí se puede desprender que Croacia y Bélgica apostaron por un juego más agresivo y con una marcación más fuerte, Francia e Inglaterra poseen una menor cantidad de infracciones, uno pensaría que Francia iba a seguir la tendencia de croatas y belgas pero fueron más eficientes con pocas faltas. Del resto se puede rescatar que Colombia fue el que más faltas ocasionó, incluyendo una expulsión, es para tener cuidado dado que es un rival directo para clasificar al mundial.

### Poder goleador

In [25]:
# Manejo de datos
info_goles = pd.DataFrame()
for j in equipos:
	partidas = equipos_2018.get_group(j).reset_index()
	partidas = partidas[['equipo','goles','intentos','arco']]
	cantidad_de_partidos = partidas.shape[0]
	total_goles = partidas.goles.sum() 
	total_intentos = partidas.intentos.sum()
	total_arco = partidas.arco.sum()
	if(cantidad_de_partidos==3):
		pos = 32
	elif(cantidad_de_partidos==4):
		pos = 16
	elif(cantidad_de_partidos==5):
		pos = 8
	elif(cantidad_de_partidos==6):
		pos = 4
	elif(cantidad_de_partidos==7):
		if(j=='France'):
			pos = 1
		elif(j=='Croatia'):
			pos = 2
		elif(j=='Belgium'):
			pos = 3
		elif(j=='England'):
			pos = 4
	partidas = partidas[['equipo']]
	fila = partidas.loc[0,:]
	fila['total_goles'] = total_goles
	fila['total_intentos'] = total_intentos
	fila['total_arco'] = total_arco
	fila['posicion'] = pos
	info_goles = info_goles.append(fila,ignore_index=True)

info_goles.loc[info_goles.posicion == 8,'posicion'] = range(5,9)
info_goles = info_goles.sort_values("posicion",ascending=True).reset_index(drop=True)
info_goles['precision_tiros'] = round(info_goles.total_arco / info_goles.total_intentos,2)

In [26]:
# Filtrar para ver los 16 primeros.
info_goles = info_goles[info_goles.posicion < 32]

In [27]:
# Visualización - Donuts
fig = {
  "data": [
    {
      "values": info_goles.total_goles.values,
      "labels": info_goles.equipo.values,
      "domain": {"x": [0, .50]},
      "name": "Goles",
      "hoverinfo":"label+name",
      "hole": 0.2,
      "type": "pie"
    },
    {
      "values": info_goles.precision_tiros.values,
      "labels": info_goles.equipo.values,
      "domain": {"x": [.51, 1]},
      "name": "Precisión en tiros",
      "hoverinfo":"label+name",
      "hole": 0.2,
      "type": "pie"
    }],
  "layout": {
        "title":"Poder goleador mundial 2018",
        "showlegend": False, # Ya se ve el país en la interacción.
        "annotations": [
            {
                "font": {
                    "size": 11
                },
                "showarrow": False,
                "text": "Goles",
                "x": 0.23,
                "y": 0.5
            },
            {
                "font": {
                    "size": 11
                },
                "showarrow": False,
                "text": "Precisión",
                "x": 0.79,
                "y": 0.5
            }
        ]
    }
}

py.iplot(fig)

#### Comentarios
En estas 2 donuts, se observa la distribución de los goles y la precisión de los tiros ( proporción de tiros que van al arco) para los 16 primeros equipos. En el donut de precisión, entre los primeros puestos están países como Rusia, Suecia y Japón. Rusia tenía la mejor precisión en tiros al arco y buena cantidad de goles considerando que no jugó todos los partidos, se tendría que ahondar más en la cantidad de goles recibidos para saber su eliminación o ver en más detalle la precisión de los tiros, ya que podrían tirar siempre donde estaba el arquero.

### Comparación con campeones pasados

In [34]:
Francia_2018 = ((df.equipo == 'France') & (df.fecha == '2018'))
Alemania_2014 = ((df.equipo == 'Germany') & (df.fecha == '2014'))
Espanna_2010 = ((df.equipo == 'Spain') & (df.fecha == '2010'))

campeones = df[Francia_2018|Alemania_2014|Espanna_2010]
campeones2 = campeones.groupby('equipo')
equipos = campeones2.groups.keys()
info_campeones = pd.DataFrame()
for champ in equipos:
    partidas = campeones2.get_group(champ).reset_index()
    partidas = partidas[['equipo','goles','intentos','arco','posesion','faltas','amarillas','rojas']]
    cantidad_partidos = partidas.shape[0]
    partidas['posesion promedio'] = round(partidas.posesion.sum()/cantidad_partidos,1)
    partidas['total faltas'] = partidas.faltas.sum()
    partidas['total amarillas'] = partidas.amarillas.sum()
    partidas['total rojas'] = partidas.rojas.sum()
    partidas['total goles'] = partidas.goles.sum()
    partidas['total intentos'] = partidas.intentos.sum()
    partidas['total arco'] = partidas.arco.sum()
    fila = partidas.loc[0,:]
    fila = fila[['equipo','total amarillas','total arco','total faltas','total goles','total intentos','posesion promedio']]# Ninguno tiene rojas
    info_campeones = info_campeones.append(fila,ignore_index=True)

campeones_numeros = info_campeones.drop(columns='equipo')
print("Máximos por columna")
print(campeones_numeros.apply(max))
info_campeones.drop(columns='equipo',inplace=True)

Máximos por columna
posesion promedio     58.3
total amarillas       12.0
total arco            71.0
total faltas          93.0
total goles           18.0
total intentos       122.0
dtype: float64


In [35]:
# Normalización para la visualización.
info_campeones["posesion promedio"] = info_campeones["posesion promedio"] / info_campeones["posesion promedio"].max()
info_campeones["total amarillas"] = info_campeones["total amarillas"] / info_campeones["total amarillas"].max()
info_campeones["total arco"] = info_campeones["total arco"] / info_campeones["total arco"].max()
info_campeones["total faltas"] = info_campeones["total faltas"] / info_campeones["total faltas"].max()
info_campeones["total goles"] = info_campeones["total goles"] / info_campeones["total goles"].max()
info_campeones["total intentos"] = info_campeones["total intentos"] / info_campeones["total intentos"].max()

In [36]:
data = [
    go.Scatterpolar(
      theta = info_campeones.columns.tolist() + [info_campeones.columns.tolist()[0]],
      r = info_campeones.values[0].tolist()+[info_campeones.values[0].tolist()[0]],
      name = 'Francia 2018'
    ),
    go.Scatterpolar(
      theta = info_campeones.columns.tolist() + [info_campeones.columns.tolist()[0]],
      r = info_campeones.values[1].tolist()+[info_campeones.values[1].tolist()[0]],
      name = 'Alemania 2014'
    ),
    go.Scatterpolar(
      theta = info_campeones.columns.tolist() + [info_campeones.columns.tolist()[0]],
      r = info_campeones.values[2].tolist()+[info_campeones.values[2].tolist()[0]],
      name = 'España 2010'
    )
]

layout = go.Layout(
  title = 'Comparación con campeones pasados',
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, 1]
    )
  ),
  showlegend = True
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

#### Comentarios

En este gráfico de radar, se comparan los 3 campeones de los últimos mundiales en base a los atributos ya vistos en las gráficas anteriores. Se puede ver que con el tiempo, la cantidad de intentos/tiros al arco y la posesión promedio del balón ha disminuido. En cambio, se ha vuelto un juego más agresivo por el aumento de cantidades amarillas. La cantidad de goles se ha mantenido en un rango normal en base a los años 2010 y 2014.

### Conclusiones y trabajo futuro
En base a los datos visualizados y analizados del último mundial de fútbol, se puede proponer que el nuevo estilo de la selección sea más táctico o esperando el contra-ataque, ya que los 4 finalistas del mundial pasado estuvieron entre un rango de 48-55% de posesión. Para esta medida, se puede fortalecer o entrenar con más intensidad el marcaje, cambiar el sistema táctico y coordinar a los atacantes para ayudar en la defensa.
En cuanto a la cantidad de faltas, no es una variable dominante para imponer un nuevo estilo, no hay un patrón claro de que si uno comete menos o más faltas logrará mejores resultados.
La precisión de los tiros ayudará al equipo pero solo si se tiene una buena defensa, ya que Rusia a pesar de tener la mejor precisión y una cantidad de goles considerable, solo menor que los 4 finalistas, fue eliminado en cuartos de final de forma prematura. 
Con el último gráfico se puede observar el nuevo cambio de paradigma en el fútbol, antes España y Alemania tenían una gran pose de balón y arriesgaban mucho al arco comparado con Francia. Para lograr ser campeón, Francia además tuvo que realizar una mayor cantidad de faltas que comprometieran una tarjeta amarilla.
Esto respalda el consejo de mantener la cautela y no ser tan protagonista en el partido, atacar en los momentos claves y cometer una baja cantidad de faltas. No será el estilo de juego más vistoso pero es el que está dando resultado el último tiempo para la contienda mundial.

En cuanto al trabajo futuro, se pueden analizar otros atributos que están en el dataset, como la precisión de los pases(¿influye realmente?), la distancia recorrida de los equipos ( ¿Se cumple que hay que correr más para lograr mejores resultados? ¿Habrá que implementar más ejercicios para aumentar la resistencia física?), ver con más detalle la colocación de los tiros al arco, mapas de calor sobre los lugares donde más se recorre en el campo, entre otros.

#### Referencias

Dataset utilizado:
['Predict FIFA 2018 Man of the Match'](https://www.kaggle.com/mathan/fifa-2018-match-statistics)    